# Typology for doubles

1. Identify 2-artifact clusters (pairs of contiguous artifacts); make sure that the union has no interior
2. For each cluster, classify the inside edge as C or nonC (`drop_interline`: bool)
3. Visualize and test if it works

- number of nodes
- number of continuity groups
- filter non-planarity artifacts
- CES counts
- prime detection
- crosses detection
- touches detecion

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import numpy as np
import pandas as pd
import shapely
from libpysal import graph
from scipy import sparse
import folium
import folium.plugins as plugins
import shapely

from core import algorithms, utils
from core.geometry import voronoi_skeleton

Specify case metadata

In [ ]:
case = "Liège"

Read road data

In [ ]:
roads = utils.read_parquet_roads(case)

Remove duplicated roads

In [ ]:
roads = momepy.remove_false_nodes(roads)
roads = roads[~roads.geometry.duplicated()].reset_index()
roads = momepy.remove_false_nodes(roads)

Assign COINS-based information to roads

In [ ]:
# %%time
roads, coins = algorithms.common.continuity(roads)

Read artifacts

In [ ]:
artifacts = momepy.FaceArtifacts(roads).face_artifacts.set_crs(roads.crs)
artifacts["id"] = artifacts.index

Remove edges fully within the artifact (dangles).

In [ ]:
a_idx, _ = roads.sindex.query(artifacts.geometry, predicate="contains")
artifacts = artifacts.drop(artifacts.index[a_idx])

Get nodes from the network.

In [ ]:
nodes = momepy.nx_to_gdf(momepy.node_degree(momepy.gdf_to_nx(roads)), lines=False)

Link nodes to artifacts

In [ ]:
node_idx, artifact_idx = artifacts.sindex.query(
    nodes.buffer(0.1), predicate="intersects"
)
intersects = sparse.coo_array(
    ([True] * len(node_idx), (node_idx, artifact_idx)),
    shape=(len(nodes), len(artifacts)),
    dtype=np.bool_,
)

Compute number of nodes per artifact

In [ ]:
artifacts["node_count"] = intersects.sum(axis=0)

Apply additional filters to remove artifacts that are not suitable for simplification. These may be artifacts that:
- are too large in size
- are part of a larger intersection that may need different methods of simplification

In [ ]:
area_threshold = 1250  # this is hard to determine but it can be done iteratively using different thresholds
circular_compactness_threshold = 0.15  # same as above

rook = graph.Graph.build_contiguity(artifacts, rook=True)

**keeping only size 2 clusters!**

In [ ]:
artifacts['comp'] = rook.component_labels
counts = artifacts['comp'].value_counts()
artifacts = artifacts.loc[artifacts['comp'].isin(counts[counts==2].index)]

Compute number of stroke groups per artifact.

In [ ]:
def _get_stroke_info(artifacts, roads):
    strokes = []
    c_ = []
    e_ = []
    s_ = []
    for geom in artifacts.geometry:
        singles = 0
        ends = 0
        edges = roads.iloc[roads.sindex.query(geom, predicate="covers")]
        if (  # roundabout special case
            edges.coins_group.nunique() == 1
            and edges.shape[0] == edges.coins_count.iloc[0]
        ):
            singles = 1
            mains = 0
        else:
            all_ends = edges[edges.coins_end]
            mains = edges[
                ~edges.coins_group.isin(all_ends.coins_group)
            ].coins_group.nunique()

            visited = []
            for coins_count, group in zip(
                all_ends.coins_count, all_ends.coins_group, strict=True
            ):
                if (group not in visited) and (
                    coins_count == (edges.coins_group == group).sum()
                ):
                    singles += 1
                    visited.append(group)
                elif group not in visited:
                    ends += 1
                    # do not add to visited as they may be disjoint within the artifact
        strokes.append(edges.coins_group.nunique())
        c_.append(mains)
        e_.append(ends)
        s_.append(singles)
    return strokes, c_, e_, s_


strokes, c_, e_, s_ = _get_stroke_info(artifacts, roads)
artifacts["stroke_count"] = strokes
artifacts["C"] = c_
artifacts["E"] = e_
artifacts["S"] = s_

Filer artifacts caused by non-planar intersections.

In [ ]:
artifacts["non_planar"] = artifacts["stroke_count"] > artifacts["node_count"]
a_idx, r_idx = roads.sindex.query(artifacts.geometry.boundary, predicate="overlaps")
artifacts.iloc[np.unique(a_idx), -1] = True

**Remove (for now) the clusters that contain at least one non-planar component** (we will deal with them later, ...?)

In [ ]:
# non_planar_cluster: number of non-planar artifacts per cluster
artifacts["non_planar_cluster"] = artifacts.apply(lambda x: sum(artifacts.loc[artifacts["comp"]==x.comp]["non_planar"]), axis = 1)
# dealing with the non-planar later!
artifacts = artifacts[artifacts.non_planar_cluster == 0]

Count intersititial nodes (primes).

In [ ]:
artifacts["interstitial_nodes"] = artifacts.node_count - artifacts[["C", "E", "S"]].sum(
    axis=1
)

Define the type label.

In [ ]:
ces_type = []
for x in artifacts[["node_count", "C", "E", "S"]].itertuples():
    ces_type.append(f"{x.node_count}{'C' * x.C}{'E' * x.E}{'S' * x.S}")
artifacts["ces_type"] = ces_type

**Classify interlines**

In [ ]:
def classify_interline(mycluster, artifacts, roads):

    # get the cluster geometry
    cluster_geom = artifacts[artifacts.comp == mycluster].union_all()

    # find the road segment that is contained within the cluster geometry
    road_contained = roads.sindex.query(cluster_geom, predicate="contains")
    
    # make sure we have uniquely identified the road segment
    assert len(road_contained)==1 

    # return the ID of the road segment (to potentially drop later) and coins_end True/False    
    return road_contained[0], roads.loc[road_contained]["coins_end"].values[0]

interlines = artifacts.comp.apply(lambda x: classify_interline(x, artifacts, roads))
artifacts["inter_road"] = [v[0] for v in interlines]
artifacts["inter_coins_end"] = [v[1] for v in interlines]
artifacts["drop_interline"] = artifacts["inter_coins_end"]

In [ ]:
# if one of the 2 artifacts is of type 0C0EnS, then no matter whether inter_coins_end is True or False! it will always be S
comps_with_s = list(artifacts[(artifacts.C == 0) & (artifacts.E == 0)].comp)
artifacts.loc[artifacts["comp"].isin(comps_with_s), "drop_interline"] = True

Visual check to see whether classification of interlines worked

In [ ]:
m = roads.explore(
    "coins_len",
    k=20,
    scheme="quantiles",
    tiles="cartodb positron",
    prefer_canvas=True,
    name="Roads",
    highlight_kwds={"color": "red"},
    style_kwds=dict(weight=5),
    max_zoom=52,
    opacity=0.5,
)
artifacts.explore(m=m, name="Artifacts", column="drop_interline")
nodes.explore(m=m, name="Nodes", color="blue", marker_size=250)
folium.LayerControl().add_to(m)
plugins.MousePosition().add_to(m)

m